In [2]:
import pandas as pd
import mysql.connector


In [3]:

def convert_table_to_df(cursor, database_name, table_name): 
    query = f"SELECT column_name FROM information_schema.columns WHERE table_schema = '{database_name}' AND table_name = '{table_name}'"
    cursor.execute(query)
    rows = cursor.fetchall()
    # get the columns
    columns =  [row[0] for row in rows]
    data_query = f"SELECT { ','.join([f'`{col}`' for col in columns]) } from `{table_name}`"
    cursor.execute(data_query)
    # get records
    data_records = cursor.fetchall()
    # print(data_records)
    dict_df = {col:[] for col in columns}
    for record in data_records:
        for idx, column in enumerate(columns):
            dict_df[column].append(record[idx])
    return pd.DataFrame(dict_df)

        

In [5]:

def convert_all_tables():
    conn = mysql.connector.connect(
        host="127.0.0.1",
        user="jcsf",
        password="hello",
        database="a2"
    )
    cursor = conn.cursor()
    cursor.execute('SHOW TABLES')
    tables = cursor.fetchall()
    tables = [table[0] for table in tables]
    for table in tables:
        df = convert_table_to_df(cursor, conn.database, table)
        df.to_csv(f"./csvs/{table}.csv")
    

In [ ]:
convert_all_tables()


In [12]:
conn = mysql.connector.connect(
        host="127.0.0.1",
        user="jcsf",
        password="hello",
        database="a2"
    )
cursor = conn.cursor()
df = convert_table_to_df(cursor, conn.database, "Person")

In [13]:
first_names = []
last_names = []
for idx, row in df.iterrows():
    name:str = row['name']
    split_name = name.rsplit(" ", maxsplit=1)
    first_name = split_name[0]
    if len(split_name) == 2:
        last_name = split_name[1]
    else:
        last_name = None
    first_names.append(first_name)
    last_names.append(last_name)

df.insert(len(df.columns), "first_name", first_names)
df.insert(len(df.columns), "last_name", last_names)
df = df.drop('name', axis=1)
df.to_csv("./Person.csv", index=False)

    